# 00 — Introduction: Estimand-First OPE

This notebook is a fast, narrative tour of CausalRL. The key idea is
**estimand-first OPE**: you declare the estimand and its assumptions, then
every estimator and report is obligated to surface diagnostics that tell you
whether the assumptions look plausible in the data.

We'll run a small synthetic bandit experiment (so we know ground truth),
inspect the report schema, and export a self-contained HTML report.

## Setup

Suggested environment:

```
pip install "causalrl[plots]"
```

(You can add `[docs]` or `[notebooks]` if you want full notebook tooling.)

In [1]:
from __future__ import annotations

from pathlib import Path

import numpy as np

import crl
from crl.benchmarks.bandit_synth import SyntheticBandit, SyntheticBanditConfig
from crl.ope import evaluate
from crl.utils.seeding import set_seed
from crl.viz import configure_notebook_display

In [2]:
set_seed(0)
np.random.seed(0)
configure_notebook_display()

print("crl", crl.__version__)

crl 0.1.0


## Quick OPE Run

We sample a logged bandit dataset, evaluate a target policy, and compare
estimators in a single call. The report is a structured object that can
serialize to a DataFrame or HTML.

In [3]:
benchmark = SyntheticBandit(SyntheticBanditConfig(seed=0))
dataset = benchmark.sample(num_samples=1_000, seed=1)
true_value = benchmark.true_policy_value(benchmark.target_policy)

report = evaluate(dataset=dataset, policy=benchmark.target_policy)
report.summary_table()

,value,stderr,ci,diagnostics,assumptions_checked,assumptions_flagged,warnings,metadata,lower_bound,upper_bound,estimator
0,-0.393095,0.038707,"(-0.4689600359090516, -0.3172303957405684)",{'overlap': {'min_behavior_prob': 0.1097201314...,"[sequential_ignorability, overlap]",[],[],"{'estimator': 'IS', 'num_samples': 1000, 'requ...",-0.468960,-0.317230,IS
1,-0.396854,0.042185,"(-0.47953558159888593, -0.3141718825523601)",{'overlap': {'min_behavior_prob': 0.1097201314...,"[sequential_ignorability, overlap]",[],[],"{'estimator': 'WIS', 'num_samples': 1000, 'req...",-0.479536,-0.314172,WIS
2,-0.423104,0.030082,"(-0.48206487086077743, -0.36414260107394003)",{'overlap': {'min_behavior_prob': 0.1097201314...,"[sequential_ignorability, overlap]",[],[],"{'estimator': 'DoubleRL', 'config': {'num_fold...",-0.482065,-0.364143,DoubleRL


## Report Schema and HTML Export

Every estimator returns the same schema: point estimate, uncertainty
(stderr/CI), diagnostics, and assumption flags. This makes it easy to compare
methods side-by-side and to automate downstream checks.

In [4]:
output_dir = Path("docs/assets/reports")
output_dir.mkdir(parents=True, exist_ok=True)
report_path = output_dir / "intro_bandit_report.html"
report.save_html(str(report_path))
report_path

PosixPath('docs/assets/reports/intro_bandit_report.html')

## Takeaways

- The report schema is standardized across estimators.
- Diagnostics make assumption violations visible early.
- HTML export creates shareable artifacts for reviews.